In [10]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
df = pd.read_csv("1976-2020-senate.csv")

In [3]:
df

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party_detailed,writein,mode,candidatevotes,totalvotes,unofficial,version,party_simplified
0,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,SAM STEIGER,REPUBLICAN,False,total,321236,741210,False,20210114,REPUBLICAN
1,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,WM. MATHEWS FEIGHAN,INDEPENDENT,False,total,1565,741210,False,20210114,OTHER
2,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,DENNIS DECONCINI,DEMOCRAT,False,total,400334,741210,False,20210114,DEMOCRAT
3,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,ALLAN NORWITZ,LIBERTARIAN,False,total,7310,741210,False,20210114,LIBERTARIAN
4,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,BOB FIELD,INDEPENDENT,False,total,10765,741210,False,20210114,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,2020,WYOMING,WY,56,83,68,US SENATE,statewide,gen,False,UNDER VOTES,NaN,False,total,6401,278503,False,20210114,OTHER
3625,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,KELLY LOEFFLER,REPUBLICAN,False,total,2194848,4483294,True,20210114,REPUBLICAN
3626,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,RAPHAEL WARNOCK,DEMOCRAT,False,total,2288446,4483294,True,20210114,DEMOCRAT
3627,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,False,DAVID A. PERDUE,REPUBLICAN,False,total,2213979,4483241,True,20210114,REPUBLICAN


In [6]:
df["writein"].nunique()

2

In [7]:
def sir(params, elections, registered_voters):
    beta, gamma = params
    s = [1 - elections[0]/registered_voters[0]]
    i = [elections[0]/registered_voters[0]]
    r = [0]
    
    for t in range(1, len(elections)):
        dt = elections.index[t] - elections.index[t-1]
        ds = -beta * s[-1] * i[-1] * dt.days
        di = (beta * s[-1] * i[-1] - gamma * i[-1]) * dt.days
        dr = gamma * i[-1] * dt.days
        
        s.append(s[-1] + ds)
        i.append(i[-1] + di)
        r.append(r[-1] + dr)
    
    return np.array(i)

In [9]:
def objective(params, elections, registered_voters):
    simulated = sir(params, elections, registered_voters)
    return np.sum((elections - simulated)**2)

In [13]:
california_dem = df[(df.state == 'CALIFORNIA') & (df.party_simplified == 'DEMOCRAT')]
initial_guess = [0.3, 0.1]
result = minimize(objective, initial_guess, args=(california_dem.candidatevotes, california_dem.totalvotes))
optimal_beta, optimal_gamma = result.x

KeyError: 0